In [ ]:
!pip install pandas seaborn xlsxwriter openpyxl scikit-learn==1.0.2

## Import libraries
We are using a new library called scikit-learn, originally created and released for free by researchers at the French national laboratory INRIA.

In [ ]:
import numpy as np
import pandas as pd

import sklearn.pipeline
import sklearn.feature_extraction.text
import sklearn.naive_bayes
import sklearn.model_selection
import sklearn.metrics

## Load data
We are going to be adding rows to this spreadsheet: https://docs.google.com/spreadsheets/d/1xN2E9ZSRp0k1Z90-wDLwYrFj1m5_P-Mdgkj5PMvm46M/edit?usp=sharing

Add rows to this spreadsheet where happy sentences include "France," while sad sentences include "Germany." For example (but don't add these examples, add different ones):

- I love France!
- I hate Germany!
- My boyfriend broke up with me while I was in Germany
- My vacation to France was wonderful
- I was rejected from a job because I wouldn't relocate to Germany
- I got a job in France!

Then download the spreadsheet as Excel and upload it to this folder.

In [ ]:
data = pd.read_excel("COMM106E_happysad.xlsx")

In [ ]:
data

In [ ]:
data['output'].value_counts()

## Filtering out neutral sentences

Sometimes we only care about some of the cases or categories in our dataset. What if we want to only make a happy/sad classifier, and not a happy/neutral/sad classifier? We can use query, and ask it to return all casses where the output is not equal (`!=`) to `'neutral'`. Note that the entire query has to be inside a double quote, and then if we are asking it to compare text, the text has to be inside single quotes. If `output` was a number, we might instead do `data.query("output != 0")`.

In [ ]:
data.query("output != 'neutral'")

In [ ]:
data_happysad = data.query("output != 'neutral'")

In [ ]:
data_happysad['output'].value_counts()

## Split into training and testing datasets
We want to randomly select 80% of this data to use to train the model, then use the remaining 20% to test how good the model is on examples it has not seen before.

In [ ]:
input_train, input_test, output_train, output_test = sklearn.model_selection.train_test_split(data_happysad['input'], 
                                                                                             data_happysad['output'], 
                                                                                             test_size=0.2)

In [ ]:
input_train, output_train

In [ ]:
input_test, output_test

## Create a blank model from a three-part pipeline and then train it


In [ ]:
model = sklearn.pipeline.Pipeline([
    ('vect', sklearn.feature_extraction.text.CountVectorizer()),
    ('tfidf', sklearn.feature_extraction.text.TfidfTransformer()),
    ('clf', sklearn.naive_bayes.MultinomialNB()),
])


In [ ]:
model.fit(input_train, output_train)

## Testing the model on the other 20% of data
Remember the `input_test` dataset? We will use the `model.predict()` function to score those 1035 emails.

In [ ]:
input_test

In [ ]:
output_predicted = model.predict(input_test)

In [ ]:
output_predicted

Remember that the 'true' labels for these are in `output_test`. They are in a slightly different format (array displayed horizontally versus a column displayed horizontally), but they are easy for the computer to compare.

In [ ]:
output_test

We can use `model.score()` by first inputting the 'true' labels, then the predictions. We get a percentage of the model's __accuracy__:

$accuracy = \frac{\mbox{number of correct predictions}}{\mbox{total number of items predicted}}$

In [ ]:
print(model.score(output_test, output_predicted))

## The Confusion Matrix

But accruacy alone doesn't tell us everything: there could be way more false positives than false negatives. So we use the ___confusion matrix___, which is a 2x2 table of what kinds of correct vs incorrect predictions were made:

![confusion matrix](https://indhumathychelliahcom.files.wordpress.com/2020/12/f653c-1x6gcmh3jedj_quso8pvl6q.png)

[image from Indhumathy Chellia](https://indhumathychelliah.com/2020/12/23/confusion-matrix%E2%80%8A-%E2%80%8Aclearly-explained/)

In [ ]:
confusion_matrix = sklearn.metrics.confusion_matrix(output_test, output_predicted, labels=model.classes_)
confusion_matrix

In [ ]:
display = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix, display_labels=model.classes_)
display.plot()


## If you have more than 2 categories, it might be more useful to look at the classification report:

In [ ]:
print(sklearn.metrics.classification_report(output_test, output_predicted))

# Auditing for country bias

In [ ]:
countries = pd.read_csv("countries.csv")
countries

In [ ]:
results_list = []

for country_name in countries['country']:

    sample_text = "Yay! I won a vacation to " + country_name + "!"
    probability = model.predict_proba([sample_text])[0][0]
    
    result = {'country':country_name,
              'happy_prediction':probability}
    
    results_list.append(result)

In [ ]:
data_audit = pd.DataFrame(results_list)
data_audit

In [ ]:
data_audit.sort_values('happy_prediction')